In [1]:
# Imports
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import random
from scipy import signal

In [2]:
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
# 1. Drop unneeded columns in frames data

# Returns the list of used columns (dropping lower joints, useless features...)
def computeUsedColumns() :
    # header computed on one frame row (skipping rows 2-6) with following replace regex
    # ([A-z]+)\s*\w+\s+[\-0-9\.]+\s+[\-0-9\.]+\s+[\-0-9\.]+\s+[\-0-9\.]+\s+[\-0-9\.]+\s+[\-0-9\.]+\s+[\-0-9\.]+\s+
    # $1\t$1Status\t$1LocX\t$1LocY\t$1LocZ\t$1OrW\t$1OrX\t$1OrY\t$1OrZ\t
    header = ["Index","Time","SkeletonId","HandLeftConfidence","HandLeftState","HandRightConfidence","HandRightState","SpineBase","SpineBaseStatus","SpineBaseLocX","SpineBaseLocY","SpineBaseLocZ","SpineBaseOrW","SpineBaseOrX","SpineBaseOrY","SpineBaseOrZ","SpineMid","SpineMidStatus","SpineMidLocX","SpineMidLocY","SpineMidLocZ","SpineMidOrW","SpineMidOrX","SpineMidOrY","SpineMidOrZ","Neck","NeckStatus","NeckLocX","NeckLocY","NeckLocZ","NeckOrW","NeckOrX","NeckOrY","NeckOrZ","Head","HeadStatus","HeadLocX","HeadLocY","HeadLocZ","HeadOrW","HeadOrX","HeadOrY","HeadOrZ","ShoulderLeft","ShoulderLeftStatus","ShoulderLeftLocX","ShoulderLeftLocY","ShoulderLeftLocZ","ShoulderLeftOrW","ShoulderLeftOrX","ShoulderLeftOrY","ShoulderLeftOrZ","ElbowLeft","ElbowLeftStatus","ElbowLeftLocX","ElbowLeftLocY","ElbowLeftLocZ","ElbowLeftOrW","ElbowLeftOrX","ElbowLeftOrY","ElbowLeftOrZ","WristLeft","WristLeftStatus","WristLeftLocX","WristLeftLocY","WristLeftLocZ","WristLeftOrW","WristLeftOrX","WristLeftOrY","WristLeftOrZ","HandLeft","HandLeftStatus","HandLeftLocX","HandLeftLocY","HandLeftLocZ","HandLeftOrW","HandLeftOrX","HandLeftOrY","HandLeftOrZ","ShoulderRight","ShoulderRightStatus","ShoulderRightLocX","ShoulderRightLocY","ShoulderRightLocZ","ShoulderRightOrW","ShoulderRightOrX","ShoulderRightOrY","ShoulderRightOrZ","ElbowRight","ElbowRightStatus","ElbowRightLocX","ElbowRightLocY","ElbowRightLocZ","ElbowRightOrW","ElbowRightOrX","ElbowRightOrY","ElbowRightOrZ","WristRight","WristRightStatus","WristRightLocX","WristRightLocY","WristRightLocZ","WristRightOrW","WristRightOrX","WristRightOrY","WristRightOrZ","HandRight","HandRightStatus","HandRightLocX","HandRightLocY","HandRightLocZ","HandRightOrW","HandRightOrX","HandRightOrY","HandRightOrZ","HipLeft","HipLeftStatus","HipLeftLocX","HipLeftLocY","HipLeftLocZ","HipLeftOrW","HipLeftOrX","HipLeftOrY","HipLeftOrZ","KneeLeft","KneeLeftStatus","KneeLeftLocX","KneeLeftLocY","KneeLeftLocZ","KneeLeftOrW","KneeLeftOrX","KneeLeftOrY","KneeLeftOrZ","AnkleLeft","AnkleLeftStatus","AnkleLeftLocX","AnkleLeftLocY","AnkleLeftLocZ","AnkleLeftOrW","AnkleLeftOrX","AnkleLeftOrY","AnkleLeftOrZ","FootLeft","FootLeftStatus","FootLeftLocX","FootLeftLocY","FootLeftLocZ","FootLeftOrW","FootLeftOrX","FootLeftOrY","FootLeftOrZ","HipRight","HipRightStatus","HipRightLocX","HipRightLocY","HipRightLocZ","HipRightOrW","HipRightOrX","HipRightOrY","HipRightOrZ","KneeRight","KneeRightStatus","KneeRightLocX","KneeRightLocY","KneeRightLocZ","KneeRightOrW","KneeRightOrX","KneeRightOrY","KneeRightOrZ","AnkleRight","AnkleRightStatus","AnkleRightLocX","AnkleRightLocY","AnkleRightLocZ","AnkleRightOrW","AnkleRightOrX","AnkleRightOrY","AnkleRightOrZ","FootRight","FootRightStatus","FootRightLocX","FootRightLocY","FootRightLocZ","FootRightOrW","FootRightOrX","FootRightOrY","FootRightOrZ","SpineShoulder","SpineShoulderStatus","SpineShoulderLocX","SpineShoulderLocY","SpineShoulderLocZ","SpineShoulderOrW","SpineShoulderOrX","SpineShoulderOrY","SpineShoulderOrZ","HandTipLeft","HandTipLeftStatus","HandTipLeftLocX","HandTipLeftLocY","HandTipLeftLocZ","HandTipLeftOrW","HandTipLeftOrX","HandTipLeftOrY","HandTipLeftOrZ","ThumbLeft","ThumbLeftStatus","ThumbLeftLocX","ThumbLeftLocY","ThumbLeftLocZ","ThumbLeftOrW","ThumbLeftOrX","ThumbLeftOrY","ThumbLeftOrZ","HandTipRight","HandTipRightStatus","HandTipRightLocX","HandTipRightLocY","HandTipRightLocZ","HandTipRightOrW","HandTipRightOrX","HandTipRightOrY","HandTipRightOrZ","ThumbRight","ThumbRightStatus","ThumbRightLocX","ThumbRightLocY","ThumbRightLocZ","ThumbRightOrW","ThumbRightOrX","ThumbRightOrY","ThumbRightOrZ"]
    cols = dict()
    for i in range(len(header)) :
        cols[i] = header[i]

    # confidence
    for i in range(2,7) :
        del cols[i]

    # joint name
    for i in range(7,224,9) :
        del cols[i]

    # 8 lower joints
    for i in [116, 125, 134, 143, 152, 161, 170, 179] :
        for j in range(8) :
            del cols[i+j]

    # always null orientations
    for i in [39, 201, 210, 219, 228] :
        for j in range(4) :
            del cols[i+j]

    # remove tracking status features
    for i in [8, 17, 26, 35, 44, 53, 62, 71, 80, 89, 98, 107, 188, 197, 206, 215, 224] :
        del cols[i]

    return cols

# Loads a skeleton dataframe from 'filepath' CSV, keeping only 'cols' columns
def loadEggnogSkeleton(filepath, cols):
    filepath = "datasets/eggnog.original/" + filepath
    dataframe = pd.read_csv(filepath, header=None, skiprows=[0], usecols=list(cols.keys()), names=list(cols.values()))
    return dataframe


def multivariate_data(dataset, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - history_size - target_size

  for i in range(start_index, end_index):
    indices = range(i, i+history_size, step)
    data.append(dataset[indices])

  return np.array(data)

def generatePredictions(eggSkel, path, cols) :
    frames = eggSkel.drop(["Index", "Time"], 1) 
    a=frames.values

    X = multivariate_data(dataset=a, start_index=0, end_index=None, history_size=20, target_size=0, step=1)

    model=tf.keras.models.load_model("/home/lepluard/denver/workspace/data")

    results=model.predict(X)

    #flattened = [val for sublist in results for val in sublist]
    flattened = [val[1] for val in results]

    flattened=[round(float(i), 3) for i in flattened]

    data=dict()

    data["data"]=flattened

    with open(path, 'w') as outfile:
        donnee=json.dumps(data, sort_keys=True)
        outfile.writelines(donnee)

In [4]:
# 2. Train & test set construction

# hyperparameters
# moveDetectionDuraction (ns) : window of time around which the frames are considered as a start/end frame [-moveDetectionDuration,moveDetectionDuration]

# Loads frames corresponding timestamps from 'filepath' CSV
def loadTimestamps(filepath):
    filepath = "datasets/eggnog.original/" + filepath
    # print(filepath)
    dataframe = pd.read_csv(filepath)
    return dataframe

def createFrameTest(eFrames, timestamps, moveDetectionDuration) :
    def getFrameLabels(frame) :
        isSF, isEF = False, False
        timestampsLen = len(timestamps)
        for f in eFrames :
            if f < timestampsLen and abs(frame[1] - timestamps[f][1]) < moveDetectionDuration :
                isSF = True
                #print(timestamps[f][1] - moveDetectionDuration, "<", frame[1], "<", timestamps[f][1] + moveDetectionDuration, "for frame",f)
                break
#        for f in eFrames :
#            if abs(frame[1] - f*1000000000/30) < moveDetectionDuration :
#                isEF = True
#                print(frame[1], "ef frame", f)
#                break
        return int(isSF)
    return getFrameLabels
    


In [5]:
# 3. Dataset creation & labels parsing

# Creates and returns the dataset as an array of [frames as numpy array, labels as list of booleans] loaded from the "Labels.tsv"
#    by matching each movement end label found with its corresponding frames (within a time interval) marking it as "True"
def generateLabelFiles(columns, moveDetectionDuration = 333333) :
    labels = pd.read_csv("datasets/eggnog/Labels.tsv", sep='\t')
    eFrames = list()
    data = dict()
    lastFile = ""
    n=0
    ns, nf = 0, 0
    diag = True
    for i in range(len(labels)) :
        l = labels.loc[i]
        n+=1
        #if n > 10000 : break
        # else : print(str(n)+"/400")
        if lastFile != "" and lastFile != l["File Name"] :
            lastFile = lastFile.replace("\\", "/")
            eggSkel = loadEggnogSkeleton(lastFile+"_Skeleton.txt", columns)
            timestampsOk = False
            try:
                timestamps = loadTimestamps(lastFile+"_RGB.frames")
                timestampsOk = True
            except FileNotFoundError:
                try:
                    timestamps = loadTimestamps(lastFile+"_Video.frames")
                    timestampsOk = True
                except FileNotFoundError:
                    print(lastFile+ " ignored, no frame file found")
                    nf +=1
            if(timestampsOk) :
                # Labels generation
                labs = list(map(createFrameTest(eFrames, timestamps.values, moveDetectionDuration), eggSkel.values))
                ex = { "data" : labs }
                ex = json.dumps(ex)
                #print(lastFile+" : "+ex)
                with open("datasets/eggnog.original/"+lastFile+"_psartek_labels.json", 'w') as outfile:
                    outfile.writelines(ex)
                
                # Predictions generation
                generatePredictions(eggSkel, "datasets/eggnog.original/"+lastFile+"_psartek_predictions.json", columns)
                
                if diag : 
                    a = np.array(labs)
                    movFramesCount = np.sum(a == True)
                    print(lastFile+" well processed: "+str(len(eFrames))+" labs => "+ str(movFramesCount)+"/"+str(len(labs))+" frames marked.")
                else :
                    print(lastFile+" well processed !")
                ns += 1
            #else :
            #    print(lastFile+" couldnt be parsed: timestamps == False")
            eFrames = list()
        lastFile = l["File Name"]
        eFrames.append(l["End Frame"])
    print(str(ns) +" files generated for "+str(nf)+" fails (frames file not found), itered on "+str(n)+" total labels")

In [6]:
cols = computeUsedColumns()
generateLabelFiles(cols)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
s01/part1_layout_p02/20151105_191251_00 well processed: 239 labs => 415/11799 frames marked.
s01/part1_layout_p02/20151105_192144_00 well processed: 16 labs => 30/511 frames marked.
s01/part1_layout_p02/20151105_192312_00 well processed: 21 labs => 38/618 frames marked.
s01/part1_layout_p02/20151105_192407_00 well processed: 23 labs => 40/816 frames marked.
s01/part1_layout_p02/20151105_192610_00 well processed: 34 labs => 60/1048 frames marked.
s01/part1_layout_p02/20151105_192717_00 well processed: 41 labs => 66/1307 fra

s05/part2_layout_p09/20151114_021757_00 ignored, no frame file found
s05/part2_layout_p09/20151114_022008_00 ignored, no frame file found
s06/part1_layout_p12/20151116_210615_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211101_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211141_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211449_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211545_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211627_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211720_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211802_00 ignored, no frame file found
s06/part1_layout_p12/20151116_211928_00 ignored, no frame file found
s06/part1_layout_p12/20151116_212006_00 ignored, no frame file found
s06/part2_layout_p11/20151116_212250_00 ignored, no frame file found
s06/part2_layout_p11/20151116_212431_00 ignored, no frame file found
s06/part2_layout_p11/20151116_2125

s09/part1_layout_p18/20151202_220510_00 well processed: 17 labs => 32/762 frames marked.
s09/part1_layout_p18/20151202_220552_00 well processed: 11 labs => 18/448 frames marked.
s09/part1_layout_p18/20151202_220620_00 well processed: 39 labs => 72/1572 frames marked.
s09/part1_layout_p18/20151202_220739_00 well processed: 7 labs => 13/218 frames marked.
s09/part1_layout_p18/20151202_220759_00 well processed: 26 labs => 46/986 frames marked.
s09/part1_layout_p18/20151202_220845_00 well processed: 51 labs => 91/1850 frames marked.
s09/part1_layout_p18/20151202_221009_00 well processed: 14 labs => 25/438 frames marked.
s09/part2_layout_p17/20151202_221250_00 well processed: 22 labs => 39/996 frames marked.
s09/part2_layout_p17/20151202_221340_00 well processed: 9 labs => 13/698 frames marked.
s09/part2_layout_p17/20151202_221422_00 well processed: 7 labs => 13/278 frames marked.
s09/part2_layout_p17/20151202_221442_00 well processed: 10 labs => 18/333 frames marked.
s09/part2_layout_p17/2

s15/part2_layout_p29/20151217_024016_00 ignored, no frame file found
s15/part2_layout_p29/20151217_024119_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194050_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194122_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194217_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194244_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194307_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194339_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194418_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194532_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194612_00 ignored, no frame file found
s17/part1_layouts_p34/20160128_194658_00 ignored, no frame file found
s17/part2_layouts_p33/20160128_195152_00 ignored, no frame file found
s17/part2_layouts_p33/20160128_195214_00 ignored, no frame file found
s17/part2_layouts_p33/